In [ ]:
# Zaimplementować algorytm klasyfikacji binarnej kNN (k najbliższych sąsiadów).
# Należy udostępnić metody train i predict.
# Train buduje bazę przypadków uczących (przyjmuje przynajmniej wektory i prawidłowe odpowiedzi).
# Wielokrotne wywołanie metody train powinno rozszerzać zbiór przypadków uczących.
# Metoda predict przyjmuje wektor (opcjonalnie: większą liczbę wektorów naraz) i zwraca odpowiedź klasyfikatora.
# Należy umożliwić wybór jednej z czterech funkcji odległości: euklidesowej, taksówkowej, maksimum i cosinusowej.

# Porównać wyniki na podanych zbiorach dla przynajmniej 3 wartości k (ostatnia kolumna zawiera etykietę).

# Pomocna może być metoda np.array.partition.
# Należy użyć biblioteki numpy.

# Termin: wt. 17.12.2024 08:00

import csv
import numpy as np
from enum import Enum

class DistanceMetric(Enum):
    EUCLIDEAN = 'euclidean'
    MANHATTAN = 'manhattan'
    CHEBYSHEV = 'chebyshev'
    COSINE = 'cosine'

class KNNClassifier:
    def __init__(self, k=3, distance_metric=DistanceMetric.EUCLIDEAN):
        if not isinstance(distance_metric, DistanceMetric):
            raise ValueError(f"Invalid distance metric. Choose from: {list(DistanceMetric)}")
        
        self.k = k
        self.distance_metric = distance_metric
        self.x_train = np.empty((0,))
        self.y_train = np.empty((0,), dtype=int)

    def train(self, x, y):
        x, y = np.array(x), np.array(y)
        if self.x_train.size == 0:
            self.x_train, self.y_train = x, y
        else:
            self.x_train = np.vstack((self.x_train, x))
            self.y_train = np.hstack((self.y_train, y))

    def _compute_distances(self, x):
        if self.distance_metric == DistanceMetric.EUCLIDEAN:
            return np.linalg.norm(self.x_train - x, axis=1)
        
        elif self.distance_metric == DistanceMetric.MANHATTAN:
            return np.sum(np.abs(self.x_train - x), axis=1)
        
        elif self.distance_metric == DistanceMetric.CHEBYSHEV:
            return np.max(np.abs(self.x_train - x), axis=1)
        
        elif self.distance_metric == DistanceMetric.COSINE:
            x_norm = np.linalg.norm(x)
            train_norms = np.linalg.norm(self.x_train, axis=1)
            ratio = np.dot(self.x_train, x) / (train_norms * x_norm)
            return 1 - ratio
        
        else:
            raise ValueError("Unsupported distance metric")

    def predict(self, x):
        if x.shape[0] != self.x_train.shape[1]:
            raise ValueError(f"Input vector dimension {x.shape[0]} does not match training data dimension {self.x_train.shape[1]}")
        
        distances = self._compute_distances(x)
        k_nearest_index = np.argpartition(distances[:, 0], self.k)[:self.k]
        labels = self.y_train[k_nearest_index]
        prediction = np.argmax(np.bincount(labels))
        return np.array(prediction)

In [2]:
def open_csv(file_path):
    x, y = [], []
    with open(file_path, 'r') as csvfile:
        csvreader = csv.reader(csvfile, delimiter=' ')
        for row in csvreader:
            x.append(list(map(float, row[:-1])))
            y.append(int(row[-1]))
    return np.array(x), np.array(y)

In [3]:
x, y = open_csv('C:/moje pliki/dokumenty i pliki/studia/magisterka_Uczenie_Maszynowe/Jezyk Python/jezyk_python_lab/dataset0.csv')       
print(x)
print(y)
# # Testowanie klasyfikatora
# knn = KNNClassifier(distance_metric='euclidean')
# knn.train(x, y)
# prediction = knn.predict(np.array([1.0, 2.0]))
# print(f'Prediction: {prediction}') # 1

# knn.train(np.array([1., 2.]), np.array(1))
# prediction = knn.predict(np.array([14.0, 23.0]))
# print(f'Prediction: {prediction}') # 1

[[0. 0.]
 [0. 1.]
 [1. 1.]
 [1. 0.]]
[0 1 2 1]


In [12]:
knn = KNNClassifier()
knn.train(x, y)
print(knn.x_train)
distances = knn._compute_distances(np.array([1,0]))
print(distances)

[[0. 0.]
 [0. 1.]
 [1. 1.]
 [1. 0.]]
[1.         1.41421356 1.         0.        ]


In [13]:
distances.shape

(4,)

In [15]:
knn.x_train.shape

(4, 2)

In [21]:
k_nearest = np.argpartition(distances, knn.k)[:knn.k]
print(k_nearest)

[0 3 2]


In [ ]:
knn.y_train

array([0, 1, 2, 1])

In [29]:
labels = knn.y_train[k_nearest]
labels

array([0, 1, 2])

In [36]:
np.argmax(np.bincount([1,1,6,3,3,0]))

1

In [37]:
np.unique([1,1,6,3,3,0], return_counts=True)

(array([0, 1, 3, 6]), array([1, 2, 2, 1], dtype=int64))

In [ ]:
x, y = open_csv('C:/moje pliki/dokumenty i pliki/studia/magisterka_Uczenie_Maszynowe/Jezyk Python/jezyk_python_lab/dataset1.csv')
# Testowanie klasyfikatora
knn = KNNClassifier(distance_metric='chebyshev')
knn.train(x, y)
prediction = knn.predict(np.array([121.314024, 222.530757]), k = 5)
print(f'Prediction: {prediction}') # 1

knn.train(np.array([121.314024, 222.530757]), np.array(1))
prediction = knn.predict(np.array([0.45540533, 6.420678]))
print(f'Prediction: {prediction}') # 0

In [ ]:



#############################################################################################################################################

x, y = open_csv('C:/moje pliki/dokumenty i pliki/studia/magisterka_Uczenie_Maszynowe/Jezyk Python/jezyk_python_lab/dataset2.csv')
# Testowanie klasyfikatora
knn = KNNClassifier(distance_metric='manhattan')
knn.train(x, y)
prediction = knn.predict(np.array([121.314024, 222.530757, 23.3578, 87.4542]))
print(f'Prediction: {prediction}') # 1

knn.train(np.array([121.314024, 222.530757, 23.3578, 87.4542]), np.array(1))
prediction = knn.predict(np.array([0.45540533, 6.420678, 86.3435, 2.3546]))
print(f'Prediction: {prediction}') # 0

x, y = open_csv('C:/moje pliki/dokumenty i pliki/studia/magisterka_Uczenie_Maszynowe/Jezyk Python/jezyk_python_lab/dataset2.csv')
# Testowanie klasyfikatora
knn = KNNClassifier(distance_metric='euclidean')
knn.train(x, y)
prediction = knn.predict(np.array([121.314024, 222.530757, 23.3578, 87.4542]))
print(f'Prediction: {prediction}') # 1

knn.train(np.array([121.314024, 222.530757, 23.3578, 87.4542]), np.array(1))
prediction = knn.predict(np.array([0.45540533, 6.420678, 86.3435, 2.3546]))
print(f'Prediction: {prediction}') # 0



In [ ]:
#############################################################################################################################################
# Sprawdzenie wyłapywania wyjątków

x, y = open_csv('C:/moje pliki/dokumenty i pliki/studia/magisterka_Uczenie_Maszynowe/Jezyk Python/jezyk_python_lab/dataset0.csv')       
knn = KNNClassifier(distance_metric='eucidean')
knn.train(x, y)
prediction = knn.predict(np.array([1.0, 2.0]))
print(f'Prediction: {prediction}')

x, y = open_csv('C:/moje pliki/dokumenty i pliki/studia/magisterka_Uczenie_Maszynowe/Jezyk Python/jezyk_python_lab/dataset0.csv')       
knn = KNNClassifier(distance_metric='eucidean')
knn.train(x, y)
knn.train(np.array([1., 2., 3., 4.]), np.array(1))
prediction = knn.predict(np.array([14.0, 23.0, 2.0]))
print(f'Prediction: {prediction}')